In [ ]:
import MySQLdb
from pandas.io import sql

conn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="openmrs") 
openmrscursor = conn.cursor() 

In [ ]:
import pandas as pd
import numpy as np

df_person = pd.read_sql('select * from `person`', con = conn)
df_person_name = pd.read_sql('select * from `person_name`', con = conn)
df_patient = pd.read_sql('select * from `patient`', con = conn)
df_patient_identifier = pd.read_sql('select * from `patient_identifier`', con = conn)

In [ ]:
# All these should be 0, if there is data, DONT PROCEED!!

print('Length - person: ', len(df_person))
print('Length - person_name: ', len(df_person_name))
print('Length - patient: ', len(df_patient))
print('Length - patient_identifier: ', len(df_patient_identifier))

In [ ]:
mimicconn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="mimic4") 
mimiccursor = mimicconn.cursor()

# mimiccursor.execute('SELECT ad.subject_id, ad.hadm_id, ad.admittime, pa.anchor_age, pa.anchor_year, DATETIME_DIFF(ad.admittime, DATETIME(pa.anchor_year, 1, 1, 0, 0, 0), YEAR) + pa.anchor_age AS age FROM admissions ad INNER JOIN patients pa ON ad.subject_id = pa.subject_id') 
# rows = mimiccursor.fetchall()

In [ ]:
df_patient_mimic = pd.read_sql('select * from patients', con = mimicconn) 
print(df_patient_mimic.head()) 
print(df_patient_mimic.dtypes)

df_admission_mimic = pd.read_sql('select * from admissions', con = mimicconn) 
print(df_admission_mimic.head())

df_adm_pat_merged = df_admission_mimic.merge(df_patient_mimic, how='inner', on='subject_id')
print(df_adm_pat_merged.head())

In [ ]:
df_adm_pat_merged = df_adm_pat_merged.astype({'anchor_year_group': 'string'}) 
df_adm_pat_merged['Age'] = (pd.DatetimeIndex(df_adm_pat_merged['admittime']).year - df_adm_pat_merged['anchor_year']) + df_adm_pat_merged['anchor_age'] 
df_adm_pat_merged['Anchorbirthyear'] = df_adm_pat_merged.apply(lambda row: row.anchor_year_group[0:4], axis=1)
df_adm_pat_merged['Anchorbirthyear']= df_adm_pat_merged['Anchorbirthyear'].astype(int)
df_adm_pat_merged['birthdate'] = df_adm_pat_merged['Anchorbirthyear'] - df_adm_pat_merged['Age'] 
df_adm_pat_merged['birthdate']= df_adm_pat_merged['birthdate'].astype(str)
df_adm_pat_merged['birthdate'] = df_adm_pat_merged.apply(lambda row: row['birthdate'] + ('0101'), axis=1) 
df_adm_pat_merged['birthdate'] = pd.to_datetime(df_adm_pat_merged['birthdate'], format = "%Y/%m/%d", yearfirst = True)
df_adm_pat_merged['birthdate_estimated'] = df_adm_pat_merged['birthdate'].notnull().map({True:1, False:0})
df_adm_pat_merged
# print(df_adm_pat.dtypes) 

In [ ]:
df_adm_pat_merged_copy =  df_adm_pat_merged.copy()
df_adm_pat_merged_copy

In [ ]:
df_adm_pat = df_adm_pat_merged_copy.sort_values(by=['admittime'], inplace=False)

In [ ]:
df_adm_pat

In [ ]:
df_adm_pat = df_adm_pat.groupby(['subject_id']).first()

In [ ]:
df_adm_pat = df_adm_pat.reset_index()

In [ ]:
subj_adm = pd.Series(df_adm_pat.birthdate.values,index=df_adm_pat.subject_id).to_dict()

In [ ]:
df_adm_pat_merged['birthdate'] = df_adm_pat_merged['subject_id'].map(subj_adm )

In [ ]:
df_adm_pat_merged

In [ ]:
df_adm_pat_merged[df_adm_pat_merged['subject_id'] == 19430048]

In [ ]:
# Calculate death date

In [ ]:
df_adm_pat_merged['death_date'] = df_adm_pat_merged['hospital_expire_flag'].copy()
df_adm_pat_merged['birthdate']= df_adm_pat_merged['birthdate'].astype(str)

In [ ]:
df1 = df_adm_pat_merged[df_adm_pat_merged['death_date'] == 1]
df2 = df_adm_pat_merged[df_adm_pat_merged['death_date'] == 0]

In [ ]:
df1['birthyear'] = df1.apply(lambda row: row.birthdate[0:4], axis=1)
df1['birthyear'] = df1['birthyear'].astype(int)
df1['death_date'] = df1['birthyear'] + df1['Age']

In [ ]:
df1['death_date'] = df1['death_date'].astype(str)
df1['death_date'] = df1.apply(lambda row: row['death_date'] + ('0101'), axis=1) 
df1['death_date'] = pd.to_datetime(df1['death_date'], format = "%Y/%m/%d", yearfirst = True)
#df1['death_date'] = df1['death_date'].astype(str)

In [ ]:
df2['death_date'] = df2['death_date'].astype(str)
df2['death_date'] = df2.apply(lambda row: row['death_date'] + ('0101'), axis=1) 
df2['death_date'] = pd.to_datetime(df1['death_date'], format = "%Y/%m/%d", yearfirst = True)
#df2['death_date'] = df2['death_date'].astype(str)

In [ ]:
df1 = df1.drop(columns = ['birthyear'])

In [ ]:
df1['deathdate_estimated'] = 1

In [ ]:
df2['deathdate_estimated'] = 0

In [ ]:
frames = [df1, df2]
df_patient_admission = pd.concat(frames) 
#https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html 

In [ ]:
df_patient_admission = df_patient_admission.reset_index(drop = True) 
df_patient_admission

In [ ]:
df_patient_admission.to_csv('df_patient_admission.csv', index = False)

In [ ]:
import uuid
df_patient_admission['uuid'] = [uuid.uuid4() for x in range(len(df_patient_admission))]

In [ ]:
df_patient_admission_copy = df_patient_admission[['subject_id', 'admittime', 'hospital_expire_flag','gender','birthdate','birthdate_estimated','death_date','deathdate_estimated', 'uuid']]

In [ ]:
df_patient_admission_copy = df_patient_admission_copy.rename({'subject_id': 'person_id', 'hospital_expire_flag': 'dead'}, axis='columns')
df_patient_admission_copy['date_created'] = '2022-01-01'

In [ ]:
# df_patient_admission_copy[df_patient_admission_copy['person_id'] == 14046553]

In [ ]:
person_table = df_patient_admission_copy.sort_values(by=['admittime'], ascending=False, inplace=False)

In [ ]:
person_table = person_table.groupby(['person_id']).first()

In [ ]:
person_table = person_table.reset_index()
person_table

In [ ]:
end = pd.to_datetime('2200-01-01')
person_table['death_date'] = person_table['death_date'].fillna(end) 
person_table['death_date'].fillna(0, inplace=True)

In [ ]:
person_table = person_table.drop(columns=['admittime'])
person_table

In [ ]:
# !pip install --user pymysql
import pymysql

# Connect to the database
connection = pymysql.connect(host='localhost',
                         user='root',
                         password='MYSQLROOTPASSWORD',
                         db='openmrs')

# create cursor
cursor=connection.cursor() 

In [ ]:
# Dealing with person table

In [ ]:
# # creating column list for insertion for person table
# cols = "`,`".join([str(i) for i in person_table.columns.tolist()])

# # Insert DataFrame recrds one by one.
# for i,row in person_table.iterrows():
#     sql = "INSERT INTO `person` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
#     try:
#         cursor.execute(sql, tuple(row))
#     except MySQLdb.OperationalError:
#         continue

#     # the connection is not autocommitted by default, so we must commit to save our changes
#     connection.commit() 

In [ ]:
sql_update_creator = 'UPDATE person SET creator=1 where person_id!=1'
cursor.execute(sql_update_creator)
sql_update_death_date = 'UPDATE person SET death_date=NULL where dead!=1'

cursor.execute(sql_update_death_date)
connection.commit()

In [ ]:
# Dealing with person_name table

In [ ]:
duplicated = person_table[['person_id', 'date_created', 'uuid']]


In [ ]:
# # creating column list for insertion for person_name table
# cols = "`,`".join([str(i) for i in duplicated.columns.tolist()])

# # Insert DataFrame recrds one by one.
# for i,row in duplicated.iterrows():
#     sql0 = "SET FOREIGN_KEY_CHECKS=0"
#     sql1 = "INSERT INTO `person_name` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
#     sql2 = "SET FOREIGN_KEY_CHECKS=1"
#     try:
#         cursor.execute(sql0)
#         cursor.execute(sql1, tuple(row))
#         cursor.execute(sql2)
#     except MySQLdb.OperationalError:
#         continue

#     # the connection is not autocommitted by default, so we must commit to save our changes
#     connection.commit() 

In [ ]:
given_name = 'UPDATE person_name SET given_name=CONCAT("S", person_id) where person_id!=1'
cursor.execute(given_name)
connection.commit()

In [ ]:
family_name = 'UPDATE person_name SET family_name= "Mimic"'
cursor.execute(family_name)
connection.commit()

In [ ]:
preferred = 'UPDATE person_name SET preferred=1 WHERE person_id!=1'
creator = 'UPDATE person_name SET creator=1 WHERE person_id!=1'
cursor.execute(preferred)
cursor.execute(creator)
connection.commit()

In [ ]:
# Dealing with patient table

In [ ]:
duplicated_patient = person_table[['person_id', 'date_created']]
duplicated_patient = duplicated_patient.rename({'person_id': 'patient_id'}, axis='columns')

In [ ]:
# # creating column list for insertion for person table
# cols = "`,`".join([str(i) for i in duplicated_patient.columns.tolist()])

# # Insert DataFrame recrds one by one.
# for i,row in duplicated_patient.iterrows():
#     sql1 = "SET FOREIGN_KEY_CHECKS=0"
#     patient = "INSERT INTO `patient` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
#     sql2 = "SET FOREIGN_KEY_CHECKS=1"
    
#     try:
#         cursor.execute(sql1)
#         cursor.execute(patient, tuple(row))
#         cursor.execute(sql2)
        
#     except MySQLdb.OperationalError:
#         continue

#     # the connection is not autocommitted by default, so we must commit to save our changes
#     connection.commit() 

In [ ]:
patient_creator = 'UPDATE patient SET creator=1 where patient_id!=1'
cursor.execute(patient_creator)
connection.commit()

In [ ]:
#dealing with patient_identifier

In [ ]:
import math
def get_checkdigit(id_without_check):

    # allowable characters within identifier
    valid_chars = "0123456789ABCDEFGHIJKLMNOPQRSTUVYWXZ_"

    # remove leading or trailing whitespace, convert to uppercase
    id_without_checkdigit = id_without_check.strip().upper()

    # this will be a running total
    sum = 0;

    # loop through digits from right to left
    for n, char in enumerate(reversed(id_without_checkdigit)):

        if not valid_chars.count(char):
            raise Exception('InvalidIDException')

        # our "digit" is calculated using ASCII value - 48
        digit = ord(char) - 48

        # weight will be the current digit's contribution to
        # the running total
        weight = None
        if (n % 2 == 0):

            # for alternating digits starting with the rightmost, we
            # use our formula this is the same as multiplying x 2 and
            # adding digits together for values 0 to 9.  Using the
            # following formula allows us to gracefully calculate a
            # weight for non-numeric "digits" as well (from their
            # ASCII value - 48).
            ## Use_sparingly: In Python 3, '/' makes floats. '//' fixes it for Python 3.
            ## For cross compatibility, simply int() the result
            ##                     VVVVVVVVVVVVV
            weight = (2 * digit) - int(digit / 5) * 9
        else:
            # even-positioned digits just contribute their ascii
            # value minus 48
            weight = digit

        # keep a running total of weights
        ## Use_sparingly: removed maths.fabs()
        ## abs() is sufficient
        sum += weight

    # avoid sum less than 10 (if characters below "0" allowed,
    # this could happen)
    sum = abs(sum) + 10

    # check digit is amount needed to reach next number
    # divisible by ten. Return an integer
    return int((10 - (sum % 10)) % 10)

In [ ]:
df_patient[['patient_id']]

In [ ]:
patient_ID_identifer =  df_patient[['patient_id']]
patient_ID_identifer

identifier_list = []
for i,row in patient_ID_identifer.iterrows():
    identifier_list.append(get_checkdigit(str(row['patient_id'])))
    
identifier_list 

In [ ]:
patient_ID_identifer['check-digit'] = identifier_list

In [ ]:
patient_ID_identifer['identifier'] = patient_ID_identifer['patient_id'].astype(str) + '-' + patient_ID_identifer['check-digit'].astype(str)

In [ ]:
patient_ID_identifer_ = patient_ID_identifer[['identifier', 'patient_id']]
patient_ID_identifer_['date_created'] = '2022-01-01'
import uuid
patient_ID_identifer_['uuid'] = [uuid.uuid4() for x in range(len(patient_ID_identifer_))]
patient_ID_identifer_

In [ ]:
# To save the data
cols = "`,`".join([str(i) for i in patient_ID_identifer_.columns.tolist()])

for i,row in patient_ID_identifer_.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    patient_identifier = "INSERT INTO `patient_identifier` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)" 
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    
    try:
        cursor.execute(sql1)
        cursor.execute(patient_identifier, tuple(row))
        cursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit() 

patient_ID_type = 'UPDATE patient_identifier SET identifier_type=1 where patient_id!=1'
location_id = 'UPDATE patient_identifier SET location_id=1 where patient_id!=1'
patient_ID_creator = 'UPDATE patient_identifier SET creator=1 where patient_id!=1'
patient_ID_preferred = 'UPDATE patient_identifier SET preferred=1 WHERE patient_id!=1'

cursor.execute(patient_ID_type)
cursor.execute(location_id)
cursor.execute(patient_ID_creator)
cursor.execute(patient_ID_preferred)

connection.commit() 